In [1]:
import pandas as pd

### The task
The system: A box containing 3 noseports. Right or left noseports offer reward at some probability (currently R/L = 0.9/0.1, 0.8/0.2, 0.7/0.3). The distribution of reward probabilities between the two ports has a 0.02 probability of switching after every trial. The center port is used to initiate a new trial.

The mouse: After the ITI, the LED for the center port turns on, at which time the mouse can poke to initiate a trial. After the center poke, the mouse has a defined period of time within which it must report its decision by poking either the left or right port, which then delivers reward with one of the above probabilities.

### Load in feature matrix containing trial history

Here we've created a dataframe where each row represents a series of 10 action-outcome pairings within a single session for each mouse. Within a given session (**Session ID**), each trial is represented in relation to the 10 previous trials.

i.e., for Mouse X on Trial 11, you can see the port selected N trials back (**N_Port**), along with the corresponding reward, ITI, and trial duration. **Port** is a binary variable (0=right, 1=left), as is **Reward** (1=reward, 0=no reward). **ITI** is the period between decision poke and initiation of a new trial (center poke) in seconds, but it should be noted that there is a minimum ITI of 1s before the system permits a new trial to begin (cued by LED on). The current port selected is represented in the **Decision** column.

Other useful columns:

**Switch** indicates whether the current decision was a 'switch' (1) from the previous decision, or a repeat of the previous decision (0)

**Higher p port** indicates whether the mouse chose the higher rewarding port for the current state of the system

**Condition** is reward probabilities for current session

**Block ID** is the block # for the current session, and **Current Block Length** corresponds to that block

In [3]:
data = pd.read_csv('/Users/celia/Dropbox (HMS)/mouse_bandit/markov_full.csv', index_col=0)
data.head(10)

,index,Mouse ID,Session ID,Trial,Block Trial,Block Reward,Port Streak,Reward Streak,10_Port,10_Reward,...,1_trialDuration,0_ITI,0_trialDuration,Decision,Switch,Higher p port,Reward,Condition,Block ID,Current Block Length
0,0,Baby,04192018_Baby,11.0,11.0,6.0,8.0,1.0,0.0,0.0,...,0.432,1.262,0.522,1.0,0.0,1.0,1.0,90-10,1.0,59.0
1,1,Baby,04192018_Baby,12.0,12.0,7.0,9.0,2.0,0.0,0.0,...,0.522,2.181,0.531,1.0,0.0,1.0,1.0,90-10,1.0,59.0
2,2,Baby,04192018_Baby,13.0,13.0,8.0,10.0,3.0,1.0,1.0,...,0.531,1.737,0.540,1.0,0.0,1.0,1.0,90-10,1.0,59.0
3,3,Baby,04192018_Baby,14.0,14.0,9.0,11.0,4.0,1.0,0.0,...,0.540,1.791,0.248,1.0,0.0,1.0,1.0,90-10,1.0,59.0
4,4,Baby,04192018_Baby,15.0,15.0,10.0,12.0,5.0,1.0,1.0,...,0.248,1.275,0.360,1.0,0.0,1.0,1.0,90-10,1.0,59.0
5,5,Baby,04192018_Baby,16.0,16.0,11.0,13.0,6.0,1.0,1.0,...,0.360,2.473,0.475,1.0,0.0,1.0,1.0,90-10,1.0,59.0
6,6,Baby,04192018_Baby,17.0,17.0,12.0,14.0,7.0,1.0,1.0,...,0.475,1.879,0.493,1.0,0.0,1.0,1.0,90-10,1.0,59.0
7,7,Baby,04192018_Baby,18.0,18.0,13.0,15.0,8.0,1.0,1.0,...,0.493,2.149,0.448,1.0,0.0,1.0,1.0,90-10,1.0,59.0
8,8,Baby,04192018_Baby,19.0,19.0,14.0,16.0,9.0,1.0,0.0,...,0.448,1.384,0.300,1.0,0.0,1.0,1.0,90-10,1.0,59.0
9,9,Baby,04192018_Baby,20.0,20.0,15.0,17.0,10.0,1.0,1.0,...,0.300,1.958,0.243,1.0,0.0,1.0,1.0,90-10,1.0,59.0


### alternatively, summary of mouse performance for each 

Here you have one row per session with similar columns, plus the following: **Markov** is a binary for whether the environment changed following a transition probability after each trial (p_change_states=0.02), which is true (1) for the vast majority of these sessions. **No. Trials, Blocks, Rewards** all summarize those stats for a single session. **p(high port)** gives the average performance of the mouse for that session (calculated as average probability mouse selected the true high port)

In [5]:
record = pd.read_csv('/Users/celia/GitHub/mouse_bandit/celia_session_record.csv', index_col=0)
record.head(10)

,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
0,celia,01112018_Baby,Baby,2018-01-11,2.0,1,0.8,0.2,NaN,NaN,597.0,9.0,431.0,0.88,2.0,1.0,40.0,40.0
1,celia,01122018_Baby,Baby,2018-01-12,2.0,1,0.2,0.8,NaN,NaN,750.0,15.0,542.0,0.86,2.0,1.0,40.0,40.0
2,celia,01152018_Baby,Baby,2018-01-15,2.0,1,0.8,0.2,NaN,NaN,800.0,13.0,591.0,0.88,2.0,1.0,40.0,40.0
3,celia,01172018_Baby,Baby,2018-01-17,2.0,1,0.2,0.8,NaN,NaN,796.0,23.0,558.0,0.84,2.0,1.0,40.0,40.0
4,celia,01192018_Baby,Baby,2018-01-19,2.0,1,0.8,0.2,NaN,NaN,715.0,15.0,498.0,0.83,2.0,1.0,40.0,40.0
5,celia,01222018_Baby,Baby,2018-01-22,2.0,1,0.7,0.3,NaN,NaN,891.0,22.0,505.0,0.73,2.0,1.0,40.0,40.0
6,celia,01242018_Baby,Baby,2018-01-24,2.0,1,0.3,0.7,NaN,NaN,855.0,18.0,522.0,0.79,2.0,1.0,40.0,40.0
7,celia,01262018_Baby,Baby,2018-01-26,2.0,1,0.7,0.3,NaN,NaN,781.0,14.0,450.0,0.74,2.0,1.0,40.0,40.0
8,celia,01302018_Baby,Baby,2018-01-30,2.0,1,0.3,0.7,NaN,NaN,852.0,14.0,530.0,0.81,2.0,1.0,40.0,40.0
9,celia,01312018_Baby,Baby,2018-01-31,2.0,1,0.3,0.7,NaN,NaN,796.0,17.0,424.0,0.59,2.0,1.0,40.0,40.0
